# Attribute-GAN

## Import Libraries

In [ ]:
import os

# environment for using multi-gpu.
os.environ["CUDA_VISIBLE_DEVICES"] = "2,3"

import numpy as np
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt

# import implementation
from env import *
from DataLoader import DataLoader
from AttGan import AttGan

In [ ]:
# construct test image
test_image = cv2.imread(TEST_IMAGE)
test_image = cv2.resize(test_image, dsize=(HEIGHT, WIDTH))
test_image = (np.float32(test_image) - 128) / 128
test_image_rgb = np.ubyte((test_image + 1) * 128)
test_image_rgb = cv2.cvtColor(test_image_rgb, cv2.COLOR_BGR2RGB)

# contruct test attributes
test_att = np.array([-1., 1., 1.], dtype=np.float32)

# print original test image
print("Initial Image:")
plt.imshow(test_image_rgb)
plt.show()

# construct data loader
dloader = DataLoader(BATCH_SIZE)
dloader.build()

# construct model
model = AttGan(eta=ETA, num_att=dloader.num_att)
model.build()

# train
for e in range(EPOCHS):
    
    global_step += 1
    
    # loss for generator and discriminator
    train_loss_g = 0.0
    train_loss_d = 0.0
    
    # batch train step
    for X_src, X_att_a, X_att_b in dloader.next_batch():
        loss_g, loss_d = model.step(X_src, X_att_a, X_att_b)
        train_loss_g += loss_g
        train_loss_d += loss_d
        
    # compute average loss
    train_loss_g /= len(dloader)
    train_loss_d /= len(dloader)
    
    print(f"Epochs {e+1}/{EPOCHS}")
    print(f"Train generator loss: {train_loss_g:.8f}")
    print(f"Train discriminator loss: {train_loss_d:.8f}")
    
    # test attgan
    test_result = model.convert(test_image.reshape(-1, *test_image.shape), test_att.reshape(-1, *test_att.shape))
    test_result = test_result.squeeze()
    
    test_result = np.ubyte((test_result + 1) * 128)
    test_result = cv2.cvtColor(test_result, cv2.COLOR_BGR2RGB)
    plt.imshow(test_result)
    plt.show()
    
    model.save("./ckpts/attgan")